# 保存和读取模型

In [1]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.0.0-rc0'

导入数据

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

## 定义一个模型

In [3]:
def create_model():
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                 loss=keras.losses.sparse_categorical_crossentropy,
                 metrics=['accuracy'])
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


## checkpoint回调

In [4]:
check_path = 'example/model.ckpt'
check_dir = os.path.dirname(check_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(check_path, 
                                                 save_weights_only=True, verbose=1)

model = create_model()
model.fit(train_images, train_labels, epochs=10,
         validation_data=(test_images, test_labels),
         callbacks=[cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 864/1000 [========================>.....] - ETA: 0s - loss: 1.8740 - accuracy: 0.3646 
Epoch 00001: saving model to example/model.ckpt
1000/1000 [==============================] - 1s 610us/sample - loss: 1.8064 - accuracy: 0.4000 - val_loss: 1.3109 - val_accuracy: 0.7010
Epoch 2/10
 672/1000 [===================>..........] - ETA: 0s - loss: 1.0717 - accuracy: 0.7054
Epoch 00002: saving model to example/model.ckpt
1000/1000 [==============================] - 0s 132us/sample - loss: 1.0281 - accuracy: 0.7120 - val_loss: 0.8995 - val_accuracy: 0.7680
Epoch 3/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.7301 - accuracy: 0.8086
Epoch 00003: saving model to example/model.ckpt
1000/1000 [==============================] - 0s 136us/sample - loss: 0.7227 - accuracy: 0.8050 - val_loss: 0.7055 - val_accuracy: 0.8080
Epoch 4/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.6185 - accuracy: 0.8263
Epoc

In [5]:
ls {check_dir}

checkpoint                      model.ckpt.index
model.ckpt.data-00000-of-00001


In [6]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Untrained model, accuracy:  6.70%


In [7]:
model.load_weights(check_path)
loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Untrained model, accuracy: 85.80%


## 设置checkpoint回调

In [8]:
check_path = 'example/cp-{epoch:04d}.ckpt'
check_dir = os.path.dirname(check_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(check_path,save_weights_only=True, 
                                                 verbose=1, period=5)  # 每5

model = create_model()
model.fit(train_images, train_labels, epochs=10,
         validation_data=(test_images, test_labels),
         callbacks=[cp_callback])

W0904 13:57:35.895171 140735620006784 callbacks.py:862] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.
W0904 13:57:36.122769 140735620006784 util.py:144] Unresolved object in checkpoint: (root).optimizer.iter
W0904 13:57:36.123466 140735620006784 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1
W0904 13:57:36.123994 140735620006784 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_2
W0904 13:57:36.124741 140735620006784 util.py:144] Unresolved object in checkpoint: (root).optimizer.decay
W0904 13:57:36.125648 140735620006784 util.py:144] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0904 13:57:36.126351 140735620006784 util.py:152] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.re

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 1s 514us/sample - loss: 1.7853 - accuracy: 0.4370 - val_loss: 1.2943 - val_accuracy: 0.6910
Epoch 2/10
1000/1000 [==============================] - 0s 105us/sample - loss: 0.9937 - accuracy: 0.7360 - val_loss: 0.8650 - val_accuracy: 0.7740
Epoch 3/10
1000/1000 [==============================] - 0s 101us/sample - loss: 0.6823 - accuracy: 0.8050 - val_loss: 0.6977 - val_accuracy: 0.8080
Epoch 4/10
1000/1000 [==============================] - 0s 106us/sample - loss: 0.5690 - accuracy: 0.8380 - val_loss: 0.6203 - val_accuracy: 0.8210
Epoch 5/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.4630 - accuracy: 0.8750
Epoch 00005: saving model to example/cp-0005.ckpt
1000/1000 [==============================] - 0s 140us/sample - loss: 0.4745 - accuracy: 0.8680 - val_loss: 0.5682 - val_accuracy: 0.8300
Epoch 6/10
1000/1000 [==============================] - 0s 121us/sample - l

In [9]:
ls {check_dir}

checkpoint                        cp-0010.ckpt.index
cp-0005.ckpt.data-00000-of-00001  model.ckpt.data-00000-of-00001
cp-0005.ckpt.index                model.ckpt.index
cp-0010.ckpt.data-00000-of-00001


载入最新版模型

In [10]:
latest = tf.train.latest_checkpoint(check_dir)
print(latest)

example/cp-0010.ckpt


In [11]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print('restored model accuracy: {:5.2f}%'.format(acc*100))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

restored model accuracy: 85.00%


## 手动保持权重

In [13]:
model.save_weights('example/manually_model.ckpt')
model = create_model()
model.load_weights('example/manually_model.ckpt')
loss, acc = model.evaluate(test_images, test_labels)
print('restored model accuracy: {:5.2f}%'.format(acc*100))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

restored model accuracy: 85.00%


## 保持整个模型

In [14]:
model = create_model()
model.fit(train_images, train_labels, epochs=10,
         validation_data=(test_images, test_labels),
         )
model.save('example/whole_model.h5')

W0904 14:01:11.430777 140735620006784 util.py:144] Unresolved object in checkpoint: (root).optimizer.iter
W0904 14:01:11.431578 140735620006784 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1
W0904 14:01:11.432193 140735620006784 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_2
W0904 14:01:11.432870 140735620006784 util.py:144] Unresolved object in checkpoint: (root).optimizer.decay
W0904 14:01:11.433488 140735620006784 util.py:144] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0904 14:01:11.434410 140735620006784 util.py:152] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/alpha/guide/checkpoints#

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 0s 416us/sample - loss: 1.7203 - accuracy: 0.4450 - val_loss: 1.2667 - val_accuracy: 0.6920
Epoch 2/10
1000/1000 [==============================] - 0s 111us/sample - loss: 0.9726 - accuracy: 0.7300 - val_loss: 0.8642 - val_accuracy: 0.7660
Epoch 3/10
1000/1000 [==============================] - 0s 106us/sample - loss: 0.6985 - accuracy: 0.8060 - val_loss: 0.7052 - val_accuracy: 0.8020
Epoch 4/10
1000/1000 [==============================] - 0s 115us/sample - loss: 0.5637 - accuracy: 0.8450 - val_loss: 0.6073 - val_accuracy: 0.8380
Epoch 5/10
1000/1000 [==============================] - 0s 107us/sample - loss: 0.4937 - accuracy: 0.8570 - val_loss: 0.5661 - val_accuracy: 0.8340
Epoch 6/10
1000/1000 [==============================] - 0s 116us/sample - loss: 0.4259 - accuracy: 0.8810 - val_loss: 0.5413 - val_accuracy: 0.8350
Epoch 7/10
1000/1000 [==============================] - 0s 120us

In [15]:
new_model = keras.models.load_model('example/whole_model.h5')
new_model.summary()

W0904 14:01:40.768548 140735620006784 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 128)               100480    
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [16]:
loss, acc = model.evaluate(test_images, test_labels)
print('restored model accuracy: {:5.2f}%'.format(acc*100))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

restored model accuracy: 85.70%


## 其他导出模型的方法

In [17]:
import time
saved_model_path = "example/saved_models_{}".format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

W0904 14:03:02.498310 140735620006784 deprecation.py:323] From <ipython-input-17-c0322370895b>:4: export_saved_model (from tensorflow.python.keras.saving.saved_model_experimental) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
W0904 14:03:02.524546 140735620006784 deprecation.py:506] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W0904 14:03:02.976335 140735620006784 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/saved_model/signature_def_utils_impl.py:253: build_tensor_info (from tensorflow.python.saved_model.utils_impl

'example/saved_models_1567576982'

In [18]:
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)
new_model.summary()

W0904 14:03:48.487690 140735620006784 deprecation.py:323] From <ipython-input-18-74d99c8332aa>:1: load_from_saved_model (from tensorflow.python.keras.saving.saved_model_experimental) is deprecated and will be removed in a future version.
Instructions for updating:
The experimental save and load functions have been  deprecated. Please switch to `tf.keras.models.load_model`.


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 128)               100480    
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [19]:
# 该方法必须先运行compile函数
new_model.compile(optimizer=model.optimizer,  # keep the optimizer that was loaded
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Evaluate the restored model.
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Restored model, accuracy: 85.70%
